## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import gmaps_api_key as g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Santarem,BR,-2.4431,-54.7083,82.53,89,75,9.22,broken clouds
1,1,Rikitea,PF,-23.1203,-134.9692,74.70,78,100,14.20,light rain
2,2,Port Alfred,ZA,-33.5906,26.8910,73.22,35,0,9.10,clear sky
3,3,Ingham,US,42.6000,-84.3833,73.17,60,0,14.97,clear sky
4,4,Dikson,RU,73.5069,80.5464,32.72,91,18,8.93,few clouds


In [3]:
# Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your preferred minimum temperature? "))
max_temp = float(input("What is your preferred maximum temperature? "))

What is your preferred minimum temperature? 75
What is your preferred maximum temperature? 90


In [4]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Santarem,BR,-2.4431,-54.7083,82.53,89,75,9.22,broken clouds
10,10,Georgetown,MY,5.4112,100.3354,85.95,74,20,3.44,few clouds
12,12,Butaritari,KI,3.0707,172.7902,80.98,80,26,14.25,light rain
16,16,Metro,ID,-5.1131,105.3067,75.25,81,13,7.87,few clouds
18,18,Kluang,MY,2.0251,103.3328,77.00,87,100,2.64,overcast clouds


In [5]:
# Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Santarem,BR,82.53,broken clouds,-2.4431,-54.7083,
10,Georgetown,MY,85.95,few clouds,5.4112,100.3354,
12,Butaritari,KI,80.98,light rain,3.0707,172.7902,
16,Metro,ID,75.25,few clouds,-5.1131,105.3067,
18,Kluang,MY,77.00,overcast clouds,2.0251,103.3328,
19,Beidao,CN,77.41,overcast clouds,34.5686,105.8933,
20,Bengkulu,ID,76.82,clear sky,-3.8004,102.2655,
21,Tambacounda,SN,78.85,scattered clouds,13.7539,-13.7586,
22,Yaan,NG,82.18,light rain,7.3833,8.5667,
23,East London,ZA,76.14,clear sky,-33.0153,27.9116,


In [8]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Santarem,BR,82.53,broken clouds,-2.4431,-54.7083,Hotel Imperador
10,Georgetown,MY,85.95,few clouds,5.4112,100.3354,Cititel Penang
12,Butaritari,KI,80.98,light rain,3.0707,172.7902,Isles Sunset Lodge
16,Metro,ID,75.25,few clouds,-5.1131,105.3067,OYO 3177 Wisma Lintang
18,Kluang,MY,77.00,overcast clouds,2.0251,103.3328,Hotel Anika
19,Beidao,CN,77.41,overcast clouds,34.5686,105.8933,Shell Tianshui Maiji District Erma Road Walkin...
20,Bengkulu,ID,76.82,clear sky,-3.8004,102.2655,Grage Hotel Bengkulu
23,East London,ZA,76.14,clear sky,-33.0153,27.9116,Tu Casa
27,Atuona,PF,76.62,scattered clouds,-9.8000,-139.0333,Villa Enata
31,Jamestown,US,75.02,few clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [10]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))